In [2]:
# %%
import pandas as pd
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np

# %%


# %%


# %%
df = pd.read_csv('./../../../data/processed/ID_Y.csv')
# df = pd.read_csv('./../../data/processed/.csv')

sales = df.loc[df.IMPORT == 0].groupby(['ID', 'YEAR'])['VART'].sum().unstack()
sales = sales.loc[sales.sum(1).sort_values().index]

logsales = np.log10(sales)
demlogsales = logsales.subtract(logsales.mean(1), axis = 0)

sizes = sales.loc[sales.sum(1).sort_values().index].sum(1)

Q = 10
parts = pd.cut(sizes.cumsum()/sizes.sum(), Q, labels = range(Q)).sort_index()

# %%
## Effective Nq
eff_nq = sales.groupby(parts).count().mean(1).round().astype(int)
eff_nq

# %%

from scipy.stats import norm
# from scipy.stats import norm
# from scipy.stats import pareto
from scipy.special import erf

#  - Lognormal clipped x > 3. 
sigma = 1.2810683494198207 # 1.3149476902828778
mu = 4.536908110675739 # 4.470439741406725
# 11.5% of guys that would be below the .3 threshold.
z = (mu - 3)/sigma
cum_th = 1 - .5*(1 + erf(z/np.sqrt(2)))

# emp_nq_med = nq.groupby(level = 0).median()
N = int(eff_nq.sum().round())

N_ = int(round(N/(1 - cum_th))) # We use a larger N ..
# From the theoretical N and the ppf we can know the theoretical quantiles
x_logn_clip3 = np.array([norm.ppf(q, mu, sigma) for q in np.arange(0, 1, 1/N_) + .5/N_])
x_logn_clip3 = x_logn_clip3[(-N - 1):-1]


T = 17

# %%


# %%

dem = logsales.sub(logsales.mean(1), axis = 0)
sample_shocks = dem.loc[dem.count(1) > 1].unstack().dropna() # (1.6 m real shocks)
micro_s = logsales.sub(logsales.mean(1), axis=0).unstack().std()
zero_shock = pd.DataFrame(pd.concat(T * [pd.Series(x_logn_clip3)], axis = 1))
zero_shock.columns = sales.columns

out_list = []

n = 5
Q = 10

for m in range(n):
    print(m)
    for s in [.02, .05, .1, .25, .5]:

#         df_bs = sales.sample(frac = .5)
#         df_bs = df_bs.loc[df_bs.sum(1).sort_values().index] #sorting

#         logsales = np.log10(sales)
        simu_shocks = pd.DataFrame((s/micro_s)*np.random.choice(sample_shocks.values, zero_shock.shape))
        simu_shocks.columns = sales.columns

        total = np.power(10, zero_shock + simu_shocks)
        base = np.power(10, zero_shock) # Is really frozen in this case
        noise = np.power(10, zero_shock + simu_shocks) - np.power(10, zero_shock)
        
        total['q'] = pd.cut(total.sum(1).cumsum(), Q, labels = range(Q))

        noise_qs = noise.groupby(total['q']).sum()
        base_qs = base.groupby(total['q']).sum()
        yqs = noise_qs + base_qs
        
        nqs = total['q'].value_counts().values
#         lognqs = np.log10(nqs)
        
        out = pd.concat([yqs.var(1), noise_qs.var(1), base_qs.var(1)], axis = 1)
        out.columns = ['yqs_var', 'noise_var', 'base_var']
        out['q'] = range(Q); out['m'] = m; out['nqs'] = nqs; out['s'] = s;

        out_list += [out]





/tmp/ipykernel_691512/1858430860.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


0
1
2
3
4


In [3]:
# %%
bs_result = pd.concat(out_list).reset_index(drop = True)
# data = bs_result.groupby(['q', 's']).agg(['mean', 'std']).drop('m', axis = 1)
data = bs_result.groupby(['q', 's']).median().drop('m', axis = 1)

def percentile_lo (x): 
    return np.percentile(x, q=10)
def percentile_hi (x): 
    return np.percentile(x, q=90)

data_m = bs_result.groupby(['q', 's']).median().drop('m', axis = 1)
data_lo = bs_result.groupby(['q', 's']).agg(percentile_lo).drop('m', axis = 1)
data_hi = bs_result.groupby(['q', 's']).agg(percentile_hi).drop('m', axis = 1)

# %%
### Save:

data

yqs_var     noise_var  base_var      nqs
q s                                                  
0 0.02  9.815793e+13  9.815793e+13       0.0  99225.0
  0.05  7.858472e+14  7.858472e+14       0.0  99228.0
  0.10  3.237388e+15  3.237388e+15       0.0  99235.0
  0.25  3.342003e+16  3.342003e+16       0.0  99201.0
  0.50  3.819193e+19  3.819193e+19       0.0  98829.0
1 0.02  6.741908e+14  6.741908e+14       0.0   3879.0
  0.05  2.766125e+15  2.766125e+15       0.0   3876.0
  0.10  1.584203e+16  1.584203e+16       0.0   3874.0
  0.25  1.499824e+17  1.499824e+17       0.0   3886.0
  0.50  1.153946e+20  1.153946e+20       0.0   3802.0
2 0.02  1.477599e+15  1.477599e+15       0.0   1320.0
  0.05  5.980864e+15  5.980864e+15       0.0   1320.0
  0.10  4.545266e+16  4.545266e+16       0.0   1320.0
  0.25  4.493363e+17  4.493363e+17       0.0   1326.0
  0.50  5.154840e+19  5.154840e+19       0.0   1396.0
3 0.02  2.977892e+15  2.977892e+15       0.0    578.0
  0.05  2.099456e+16  2.099456e+16       0.0    578.0
  0.10  1.204840e+17  1.204840e+17       0.0    577.0
  0.25  9.947849e+17  9.947849e+17       0.0    581.0
  0.50  8.253678e+19  8.253678e+19       0.0    605.0
4 0.02  6.420718e+15  6.420718e+15       0.0    280.0
  0.05  4.056965e+16  4.056965e+16       0.0    280.0
  0.10  1.820845e+17  1.820845e+17       0.0    280.0
  0.25  1.890621e+18  1.890621e+18       0.0    281.0
  0.50  6.085314e+19  6.085314e+19       0.0    333.0
5 0.02  1.892129e+16  1.892129e+16       0.0    142.0
  0.05  9.771368e+16  9.771368e+16       0.0    141.0
  0.10  2.300623e+17  2.300623e+17       0.0    141.0
  0.25  3.960693e+18  3.960693e+18       0.0    142.0
  0.50  3.493165e+20  3.493165e+20       0.0    147.0
6 0.02  2.397718e+16  2.397718e+16       0.0     71.0
  0.05  1.357090e+17  1.357090e+17       0.0     72.0
  0.10  6.382404e+17  6.382404e+17       0.0     72.0
  0.25  7.679627e+18  7.679627e+18       0.0     73.0
  0.50  2.536616e+20  2.536616e+20       0.0     84.0
7 0.02  6.785167e+16  6.785167e+16       0.0     35.0
  0.05  2.561841e+17  2.561841e+17       0.0     35.0
  0.10  1.371999e+18  1.371999e+18       0.0     35.0
  0.25  1.299961e+19  1.299961e+19       0.0     36.0
  0.50  3.415847e+20  3.415847e+20       0.0     40.0
8 0.02  1.601603e+17  1.601603e+17       0.0     16.0
  0.05  5.747629e+17  5.747629e+17       0.0     16.0
  0.10  3.031269e+18  3.031269e+18       0.0     15.0
  0.25  2.561963e+19  2.561963e+19       0.0     16.0
  0.50  4.472850e+20  4.472850e+20       0.0     19.0
9 0.02  2.635303e+17  2.635303e+17       0.0      6.0
  0.05  2.062877e+18  2.062877e+18       0.0      6.0
  0.10  7.816481e+18  7.816481e+18       0.0      6.0
  0.25  4.465101e+19  4.465101e+19       0.0      6.0
  0.50  1.074772e+21  1.074772e+21       0.0      8.0